In [1]:
import numpy as np
import pickle
import cv2
import glob

from scipy.ndimage.measurements import label
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

%matplotlib inline

from vehicle_detection_utilities import *

# Read in image similar to one shown above 
image = mpimg.imread('test_image.jpg')
heat = np.zeros_like(image[:,:,0]).astype(np.float)

#this is how to call the it..
#get_feature_vector(i, **search_win_params)
def get_feature_vector(image_filename, **parameters):
    return get_img_features(image_filename, **parameters)
    
search_win_params_classify = {
'color_space' : 'YUV',
'orient' : 11,
'pix_per_cell' : 16,
'cell_per_block' : 2,
'hog_channel' : "ALL",
'fromFile' : False
}

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='YUV', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = get_feature_vector(test_img, **search_win_params_classify)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)

#GENERATES WINDOWS
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [2]:
from collections import deque
from scipy.ndimage.measurements import label

class DetectorMediator:
    def __init__(self,image):        
        
        self.boxes_que = deque(maxlen=5)
        self.heatmap = None
        self.image = image
    
    def add_heat(self, heatmap, bbox_list):
        # Iterate through list of bboxes
        for box in bbox_list:
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1            
        return heatmap
    
    def apply_threshold(self, threshold):
        
        checkmap = np.copy(self.heatmap)
        checkmap[checkmap <= threshold] = 0        
        return checkmap
    
    def generate_heatmap(self):
        '''
        Generates the heatmap
        
        Doesn't return anything because it updates an instance var.
        
        This may affect runtime
        '''
        self.heatmap = np.zeros_like(self.image[:,:,0]).astype(np.float64)
        for boxCoordList in self.boxes_que:
            self.heatmap = self.add_heat(self.heatmap, boxCoordList)
    
    def draw_labeled_bboxes(self, img, labels):
        # Iterate through all detected cars
        for car_number in range(1, labels[1]+1):
            # Find pixels with each car_number label value
            nonzero = (labels[0] == car_number).nonzero()
            # Identify x and y values of those pixels
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            # Define a bounding box based on min/max x and y
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            # Draw the box on the image
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
        # Return the image
        return img
        
    def detect_vehicles(self, input_image, box_list):
        
        self.image = input_image
        
        self.boxes_que.append(box_list)
        self.generate_heatmap()
        
        thresholded_img = self.apply_threshold( 4 )
        
        labels = label(thresholded_img)
        
        output_image = self.draw_labeled_bboxes(self.image, labels)
        
        return output_image
        


In [3]:
frameno = 0
frameno = (frameno + 1) % 5

def window_dim(output_img, frameno):
    
    w_dim = 64
    if   frameno % 5 == 0:
        w_dim = 64
    elif frameno % 4 == 0:
        w_dim = 64
    elif frameno % 3 == 0:
        w_dim = 64
    elif frameno % 2 == 0:
        w_dim = 64
    elif frameno % 1 == 0:
        w_dim = 64
    return slide_window(output_img, x_start_stop=[None, None], y_start_stop=[475, 700], 
                    xy_window=(w_dim, w_dim), xy_overlap=(0.85,.85)) 

search_win_params = {
'color_space' : 'YUV',
'orient' : 11,
'pix_per_cell' : 16,
'cell_per_block' : 2,
'hog_channel' : "ALL"
}


In [4]:
images = glob.glob('./test_pipeline/*.jpg')
frameno = 0
svc = pickle.load( open( "svc_long_features.p", "rb" ))
X_scaler = pickle.load( open("scaler_long_features.p", "rb" ))

sampleimg = mpimg.imread("./test_pipeline/frame707.jpg")
det = DetectorMediator(sampleimg)

search_win_params_classify23 = {
'color_space' : 'YUV',
'orient' : 11,
'pix_per_cell' : 16,
'cell_per_block' : 2,
'hog_channel' : "ALL",

}

output_list = []
def pipeline(image):
    
    image = image
    
    #get the sliding windows
    currentWindows = window_dim(image, frameno)
    
    #Indices we think are cars:
    box_list = search_windows(image, currentWindows, svc, X_scaler,**search_win_params)
    
    output_image = det.detect_vehicles(image, box_list)
    return output_image

count = 0
test_images = ['./test_images/test1.jpg','./test_images/test2.jpg','./test_images/test3.jpg','./test_images/test4.jpg','./test_images/test5.jpg','./test_images/test6.jpg']
draw_images = []

test_image = glob.glob('./test_images/*.jpg')

for test_image in test_images:
    test_img = mpimg.imread(test_image)
    currentWindows = window_dim(test_img, frameno)
    
    #Indices we think are cars:
    box_list = search_windows(test_img, currentWindows, svc, X_scaler,**search_win_params_classify23)
    
    image_with_boxes = draw_boxes(test_img, box_list)
    
    draw_images.append(image_with_boxes)

    
    


In [5]:
plt.imshow(output_list[6])

IndexError: list index out of range